In [1]:
!pip3 install pandas
!pip3 install PyArrow 

In [9]:
%env S3ADRESS=url
%env PASSWORD_SQL=PASSWORD
%env USER_SQL=USER

env: S3ADRESS=s3a://daniel-3cco-poupay-poupay-bucket-estruturados
env: PASSWORD_SQL=PASSWORD
env: USER_SQL=USER


In [10]:
import os

S3_address = os.environ['S3ADRESS']
password_sql = os.environ['PASSWORD_SQL']
user_sql = os.environ['USER_SQL']

In [12]:
#iniciar spark
from pyspark import SparkConf
from pyspark.sql import SparkSession
import pyspark.pandas as ps

conf = SparkConf()
#conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2')
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2,com.microsoft.azure:spark-mssql-connector_2.12:1.2.0')
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')
spark = SparkSession.builder.config(conf=conf).config("spark.driver.memory", "15g").getOrCreate()

:: loading settings :: url = jar:file:/usr/local/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.microsoft.azure#spark-mssql-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-bd0dc99f-27d7-4c61-a74e-b24b01c38dc2;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.563 in central
	found com.microsoft.azure#spark-mssql-connector_2.12;1.2.0 in central
	found com.microsoft.sqlserver#mssql-jdbc;8.4.1.jre8 in central
:: resolution report :: resolve 553ms :: artifacts dl 13ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.563 from central in [default]
	com.microsoft.azure#spark-mssql-connector_2.12;1.2.0 from central in [default]
	com.microsoft.sqlserver#mssql-jdbc;8.4.1.jre8 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.2 from central in [default]
	-------------------------------------------

In [14]:
df = spark.read.option('delimiter', ',')\
    .option('header', 'true')\
    .csv(f'{S3_address}/dados-brutos/Value_of_Production_E_All_Data_(Normalized).csv')

In [15]:
df.show()

+---------+-----------+---------+-------------------+------------+--------------------+---------+----+-----------+------------+----+
|Area Code|       Area|Item Code|               Item|Element Code|             Element|Year Code|Year|       Unit|       Value|Flag|
+---------+-----------+---------+-------------------+------------+--------------------+---------+----+-----------+------------+----+
|        2|Afghanistan|      221|Almonds, with shell|         152|Gross Production ...|     1976|1976|1000 Int. $|49550.000000|  Fc|
|        2|Afghanistan|      221|Almonds, with shell|         152|Gross Production ...|     1977|1977|1000 Int. $|45505.000000|  Fc|
|        2|Afghanistan|      221|Almonds, with shell|         152|Gross Production ...|     1978|1978|1000 Int. $|60673.000000|  Fc|
|        2|Afghanistan|      221|Almonds, with shell|         152|Gross Production ...|     1979|1979|1000 Int. $|53089.000000|  Fc|
|        2|Afghanistan|      221|Almonds, with shell|         152|Gro

In [16]:
dataframe_brazil = df.filter(df.Area == 'Brazil').toPandas()

In [17]:
dataframe_brazil.loc[dataframe_brazil['Item'] == 'Meat, cattle','Item'] = 'Carne, Gado'
dataframe_brazil.loc[dataframe_brazil['Item'] == 'Soybeans','Item'] = 'Soja'
dataframe_brazil = dataframe_brazil[dataframe_brazil['Item'].isin(['Carne, Gado','Soja'])]

In [18]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, FloatType

dataframe_brazil_pyspark = spark.createDataFrame(dataframe_brazil[['Area','Item','Year','Value']])
_df = dataframe_brazil_pyspark.select(
    col('Area').alias('AREA'),
    col('Item').alias('ITEM'),
    col('Year').cast(IntegerType()).alias('YEAR'),
    col('Value').cast(FloatType()).alias('VALUE')
)

_df.write \
   .mode('overwrite') \
   .csv(f'{S3_address}/dados-tratados/brazil_tratados_from_world_agriculture')

22/05/31 00:43:16 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/05/31 00:43:17 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/05/31 00:43:17 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.


In [12]:
dataframe = spark.read\
            .csv(f"{S3_address}/dados-tratados/brazil_tratados_from_world_agriculture")

In [14]:
dataframe = dataframe.withColumnRenamed("_c0", "Regiao")
dataframe = dataframe.withColumnRenamed("_c1", "Produto")
dataframe = dataframe.withColumnRenamed("_c2", "Ano")
dataframe = dataframe.withColumnRenamed("_c3", "valor $")
dataframe.filter(col("Produto") == "Soja").show()

+------+-------+----+---------+
|Regiao|Produto| Ano|  valor $|
+------+-------+----+---------+
|Brazil|   Soja|1961| 103834.0|
|Brazil|   Soja|1962| 132017.0|
|Brazil|   Soja|1963| 123503.0|
|Brazil|   Soja|1964| 116612.0|
|Brazil|   Soja|1965| 200096.0|
|Brazil|   Soja|1966| 227557.0|
|Brazil|   Soja|1967| 273694.0|
|Brazil|   Soja|1968| 250314.0|
|Brazil|   Soja|1969| 404114.0|
|Brazil|   Soja|1970| 576963.0|
|Brazil|   Soja|1971| 794490.0|
|Brazil|   Soja|1972|1232541.0|
|Brazil|   Soja|1973|1916763.0|
|Brazil|   Soja|1974|3012490.0|
|Brazil|   Soja|1975|3783722.0|
|Brazil|   Soja|1976|4293973.0|
|Brazil|   Soja|1977|4785931.0|
|Brazil|   Soja|1978|3648930.0|
|Brazil|   Soja|1979|3916551.0|
|Brazil|   Soja|1980|5796554.0|
+------+-------+----+---------+
only showing top 20 rows



In [15]:
from pyspark.sql.functions import col
from pyspark.sql.types import DecimalType

_df = dataframe.select(
    col('Regiao').alias('regiao'),
    col('valor $').cast(DecimalType(10,3)).alias('valor'),
    col('produto').alias('produto'),
    col('ano').cast(IntegerType()).alias('ano')
)
_df
_df.write \
   .mode('overwrite') \
   .csv(f'{S3_address}/dados-clientes/brazil_tratados_from_world_agriculture')

_df.write \
 .format('com.microsoft.sqlserver.jdbc.spark') \
 .mode('overwrite') \
 .option('driver', 'com.microsoft.sqlserver.jdbc.SQLServerDriver') \
 .option('url', 'jdbc:sqlserver://poupay-grupo05.database.windows.net;databaseName=poupay;') \
 .option('dbtable', 'tblSoybeanData') \
 .option('user', user_sql ) \
 .option('password', password_sql) \
 .save()

DataFrame[regiao: string, valor: decimal(10,3), produto: string, ano: int]